In [8]:
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.feature_selection import VarianceThreshold
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from torch.optim.lr_scheduler import ReduceLROnPlateau

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F 
from torch.utils.data import Dataset, DataLoader,TensorDataset

In [9]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print(device)
print(torch.cuda.get_device_name(0))
print(torch.cuda.memory_summary())

cuda
NVIDIA GeForce RTX 4070 Laptop GPU
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |  63915 KiB | 148046 KiB |   7898 GiB |   7898 GiB |
|       from large pool |  62792 KiB | 144049 KiB |   7516 GiB |   7516 GiB |
|       from small pool |   1123 KiB |   5533 KiB |    381 GiB |    381 GiB |
|---------------------------------------------------------------------------|
| Active memory         |  63915 KiB | 148046 KiB |   7898 GiB |   7898 GiB |
|       from large pool 

In [10]:
def get_SNP(data):   
    df = pd.read_csv(f"data/{data}")
    X = df.drop(["Phenotype", "Genotype"],axis=1) # All SNP columns
    y = df['Phenotype']  # Convert phenotype to float

    variances = X.var(axis=0)
    optimal_threshold = variances.quantile(0.25)
    var_thresh = VarianceThreshold(threshold=optimal_threshold) 
    X_var_filtered = pd.DataFrame(var_thresh.fit_transform(X), columns=X.columns[var_thresh.get_support()])
    n_features = X_var_filtered.shape[1]

    X_var_filtered = np.array(X_var_filtered)
    y = np.array(y)

    # Train-Test split (80% train, 20% test)
    X_train, X_test, y_train, y_test = train_test_split(
        X_var_filtered, y, test_size=0.1)

    # Further split train into train/val (80/20 split of the train set)
    X_train, X_val, y_train, y_val = train_test_split(
        X_train, y_train, test_size=0.1)
    
    input_dim = X_train.shape[1]

    class SNPPhenotypeDataset(Dataset):
        def __init__(self, X, y):
            self.X = torch.tensor(X, dtype=torch.float32)  
            self.y = torch.tensor(y, dtype=torch.float32).view(-1, 1)  

        def __len__(self):
            return len(self.X)

        def __getitem__(self, idx):
            return self.X[idx].unsqueeze(0), self.y[idx]
        
    train_dataset = SNPPhenotypeDataset(X_train, y_train)
    val_dataset   = SNPPhenotypeDataset(X_val, y_val)
    test_dataset  = SNPPhenotypeDataset(X_test, y_test)

    batch_size = 8

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    test_loader  = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    return train_loader, val_loader, test_loader,input_dim

In [ ]:
def createCNNmodel(blocks,hidden_units, drop,lr,decay,batch=True,activation_type="relu",optimizer_type="adam"):

    class CNNRegressor(nn.Module):
        def __init__(self, blocks,hidden_units, drop, activation_type, input_length):
            super(CNNRegressor, self).__init__()

            self.nCons = len(blocks)
            self.nLayers = len(hidden_units)

            self.activations = {
                'relu': F.relu,              
                'leaky_relu': F.leaky_relu,  
                'elu': F.elu,
                "tanh": F.tanh       
                }
            
            self.activation = self.activations.get(activation_type.lower(), F.relu)
            self.dropout = nn.Dropout(drop)
            
            #CNN part
            self.cnn_layers = nn.ModuleDict()
            self.batch_con = nn.ModuleDict()
            self.pools = nn.ModuleDict()

            self.cnn_layers["input"] = nn.Conv1d(in_channels=1, out_channels=blocks[0], kernel_size=3, padding=1)
            self.batch_con["input"] = nn.BatchNorm1d(blocks[0])
            self.pools["input"] = nn.MaxPool1d(kernel_size=2)


            for j in range(self.nCons - 1):
                self.cnn_layers[f"hidden{j}"] = nn.Conv1d(in_channels=blocks[j], out_channels=blocks[j+1], kernel_size=3, padding=1)
                self.batch_con[f"hidden{j}"] = nn.BatchNorm1d(blocks[j+1])
                self.pools[f"hidden{j}"] = nn.MaxPool1d(kernel_size=2)
                
            
            self.cnn_layers["output"] = nn.Conv1d(in_channels=blocks[-1], out_channels=blocks[-1],kernel_size=3, padding=1 )


            output_length = input_length
            # Account for each pooling layer
            for _ in range(self.nCons):
                output_length = output_length // 2
            
            flattened_size = blocks[-1] * output_length
            
            
            #Dense part
            
            self.ffn_layers = nn.ModuleDict()
            self.batch_lin = nn.ModuleDict()

            self.ffn_layers["input"] = nn.Linear(flattened_size,hidden_units[0])
            self.batch_lin["input"] = nn.BatchNorm1d(hidden_units[0])

            for i in range(self.nLayers - 1):
                self.ffn_layers[f"hidden{i}"] = nn.Linear(hidden_units[i], hidden_units[i+1])
                self.batch_lin[f"hidden{i}"] = nn.BatchNorm1d(hidden_units[i+1])

            self.ffn_layers["output"] = nn.Linear(hidden_units[-1], 1)



        def forward(self,x):

            #CNN forward pass
            x = self.cnn_layers['input'](x)
            if batch:
                x = self.batch_con["input"](x)
            x = self.activation(x)
            x = self.pools["input"](x)

            for j in range(self.nCons - 1):
                x = self.cnn_layers[f'hidden{j}'](x)
                if batch:
                    x = self.batch_con[f'hidden{j}'](x)
                x = self.activation(x)
                x = self.pools[f'hidden{j}'](x)
                
            
            x = self.cnn_layers["output"](x)

            # Dense forward pass
            x = torch.flatten(x,start_dim=1)

            x = self.ffn_layers["input"](x)
            if batch:
                x = self.batch_lin["input"](x)
            x = self.activation(x)

            for i in range(self.nLayers - 1):
                x = self.ffn_layers[f'hidden{i}'](x)
                if batch:
                    x = self.batch_lin[f'hidden{i}'](x)
                x = self.activation(x)
                x = self.dropout(x)

            # return output layer
            x = self.ffn_layers['output'](x)
            return x

    # Instantiate model
    input_length = input_dim  # D, number of features after feature selection

    model = CNNRegressor(blocks,hidden_units, drop, activation_type, input_length)

    criterion = nn.MSELoss()
    optimizers = {
    'adam': lambda: torch.optim.Adam(model.parameters(), lr=lr, weight_decay=decay),
    'sgd': lambda: torch.optim.SGD(model.parameters(), lr=lr, weight_decay=decay, momentum=0.9),
    "adamw": lambda: torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=decay),
    "rmsprop": lambda: torch.optim.RMSprop(model.parameters(), lr=lr, weight_decay=decay, momentum=0.9),
    }
    optimizer = optimizers.get(optimizer_type.lower(), optimizers['adam'])()

    return model, criterion, optimizer

In [12]:
def trainAndEval(model,criterion,optimizer,train_loader,val_loader,max_epochs=1000,patience=500):
    best_val_loss = float("inf")
    train_losses = []
    val_losses = []
    model.to(device)
    scheduler = ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=5)

    # Training loop
    for epoch in range(max_epochs):
        model.train()
        running_train_loss = 0.0
        
        for batch_X, batch_y in train_loader:
            batch_X, batch_y = batch_X.to(device), batch_y.to(device).view(-1, 1)  # Reshape y to match output

            optimizer.zero_grad()
            outputs = model(batch_X)
            loss = criterion(outputs, batch_y)
            loss.backward()
            optimizer.step()

            running_train_loss += loss.item() * batch_X.size(0)

        epoch_train_loss = running_train_loss / len(train_loader.dataset)
        train_losses.append(epoch_train_loss)

        # Validation
        model.eval()
        running_val_loss = 0.0
        with torch.no_grad():
            for batch_X, batch_y in val_loader:
                batch_X, batch_y = batch_X.to(device), batch_y.to(device).view(-1, 1)  # Reshape y to match output
                outputs = model(batch_X)
                loss = criterion(outputs, batch_y)
                running_val_loss += loss.item() * batch_X.size(0)

        epoch_val_loss = running_val_loss / len(val_loader.dataset)
        val_losses.append(epoch_val_loss)
        
        # Update learning rate based on validation loss
        scheduler.step(epoch_val_loss)

        #print(f"Epoch [{epoch+1}/{max_epochs}] | Train Loss: {epoch_train_loss:.4f} | Val Loss: {epoch_val_loss:.4f}")

        # Early Stopping Check
        if epoch_val_loss < best_val_loss:
            best_val_loss = epoch_val_loss
            best_model_state = model.state_dict()
            epochs_no_improve = 0
        else:
            epochs_no_improve += 1

        if epochs_no_improve >= patience:
            #print(f"Early stopping triggered after {epoch+1} epochs")
            #print(f"Final Train loss: {train_losses[-1]:.4f} | Final Val Loss: {val_losses[-1]:.4f}")
            break

    # Load best model for evaluation
    if 'best_model_state' in locals():
        model.load_state_dict(best_model_state)
    
    return model

In [13]:
def grid_search_CNN(target_r2=0.7, data_folder="data"):
    # Create a DataFrame to store best results for each dataset
    best_results_df = pd.DataFrame(columns=[
        'Dataset', 'MSE', 'MAE', 'R2', 'Units', 'Convolutions', 'Dropout', 'Learning_Rate', 
        'Weight_Decay', 'Epochs', 'Activation', 'Optimizer', 'Patience'
    ])
    
    # Iterate through each dataset in the folder
    for data_file in os.listdir(data_folder):
        if not data_file.endswith('.csv'):
            continue
            
        print(f"\n\n{'='*50}")
        print(f"Processing dataset: {data_file}")
        print(f"{'='*50}")
        
        # Create a DataFrame to store all trial results for this dataset
        all_trials_df = pd.DataFrame(columns=[
            'Trial', 'MSE', 'MAE', 'R2', 'Convolutions', 'Units', 'Dropout', 'Learning_Rate', 
            'Weight_Decay', 'Epochs', 'Activation', 'Optimizer', 'Patience'
        ])
        
        try:
            train_loader, val_loader, test_loader, input_dim = get_SNP(data_file)
            
            # Define parameter grid
            param_grid = {
                "NUM_EPOCH": [250, 350, 450],
                "NUM_UNITS": [[48, 24, 16, 8],
                              [32, 16, 16, 8], 
                             [16, 16, 16, 16]], 
                "DROP": [0.1, 0.2, 0.3],                       
                "LR": [0.001, 0.0005, 0.0001],                       
                "DECAY": [1e-4, 1e-5],                 
                "PATIENCE": 20,
                "ACTIVATION": ["relu", "leaky_relu", "tanh", "elu"],
                "OPTIMIZER": ["adam", "sgd", "adamw", "rmsprop"],
                "CONVOLUTIONS": [[32,16,16], [32,16,8], [16,16,16]]    
            }
            
            # Calculate total number of combinations
            total_combinations = (
                len(param_grid["NUM_UNITS"]) * 
                len(param_grid["DROP"]) *
                len(param_grid["LR"]) * 
                len(param_grid["DECAY"]) *
                len(param_grid["NUM_EPOCH"]) *
                len(param_grid["ACTIVATION"]) *
                len(param_grid["OPTIMIZER"]) *
                len(param_grid["CONVOLUTIONS"])

            )
            
            print(f"Total parameter combinations to try: {total_combinations}")
            
            best_r2 = -float('inf')
            best_params = None
            best_model = None
            best_metrics = None
            current_trial = 0
            
            # Iterate through all parameter combinations
            for epochs in param_grid["NUM_EPOCH"]:
                for blocks in param_grid["CONVOLUTIONS"]:
                    for units in param_grid["NUM_UNITS"]:
                        for act in param_grid["ACTIVATION"]:
                            for opt in param_grid["OPTIMIZER"]:
                                for drop in param_grid["DROP"]:
                                    for lr in param_grid["LR"]:
                                        for decay in param_grid["DECAY"]:
                                            current_trial += 1
                                            
                                            current_params = {
                                                "num_epochs": epochs,
                                                'num_units': units,
                                                'drop': drop,
                                                'lr': lr,
                                                'decay': decay,
                                                "acts": act,
                                                "opts": opt,
                                                "patience": param_grid["PATIENCE"],
                                                "blocks": blocks
                                            }
                                            
                                            print(f"\nTrial {current_trial}/{total_combinations}")
                                            print("Current parameters:", current_params)
                                            
                                            # Create and train model
                                            model, criterion, optimizer = createCNNmodel(
                                                hidden_units=units,
                                                blocks=blocks,
                                                drop=drop,
                                                lr=lr, 
                                                decay=decay,
                                                optimizer_type=opt,
                                                activation_type=act,
                                                input_length=input_dim
                                            )
                                            
                                            # Train model and get metrics
                                            trained_model = trainAndEval(
                                                model, criterion, optimizer, 
                                                max_epochs=epochs, 
                                                patience=param_grid["PATIENCE"],
                                                train_loader=train_loader,
                                                val_loader=val_loader
                                            )
                                            
                                            # Get predictions on test set
                                            model.eval()
                                            with torch.no_grad():
                                                all_preds = []
                                                all_targets = []
                                                for batch_X, batch_y in test_loader:
                                                    batch_X = batch_X.to(device)
                                                    outputs = model(batch_X)
                                                    all_preds.append(outputs.cpu().numpy())
                                                    all_targets.append(batch_y.numpy())
                                                    
                                            y_pred = np.concatenate(all_preds).reshape(-1)
                                            y_true = np.concatenate(all_targets).reshape(-1)
                                            
                                            # Calculate metrics
                                            mse = mean_squared_error(y_true, y_pred)
                                            mae = mean_absolute_error(y_true, y_pred)
                                            r2 = r2_score(y_true, y_pred)
                                            
                                            print(f"R2 Score: {r2:.4f}")
                                            print(f"MSE Score: {mse:.4f}")
                                            print(f"MAE Score: {mae:.4f}")
                                            
                                            # Add results to the dataset's trial DataFrame
                                            trial_results = {
                                                'Trial': current_trial,
                                                'MSE': mse,
                                                'MAE': mae,
                                                'R2': r2,
                                                'Units': str(units),
                                                'Dropout': drop,
                                                'Learning_Rate': lr,
                                                'Weight_Decay': decay,
                                                'Epochs': epochs,
                                                'Activation': act,
                                                'Optimizer': opt,
                                                'Patience': param_grid["PATIENCE"],
                                                'Convolutions': blocks
                                            }
                                            all_trials_df = pd.concat([all_trials_df, pd.DataFrame([trial_results])], ignore_index=True)
                                            
                                            # Save current trial results to CSV (append mode)
                                            dataset_name = data_file.split('.')[0]
                                            trials_csv_path = f"CNN_results/grid_search_{dataset_name}.csv"
                                            
                                            # Create directory if it doesn't exist
                                            os.makedirs("CNN_results", exist_ok=True)
                                            
                                            # Save with header only if file doesn't exist
                                            all_trials_df.iloc[-1:].to_csv(
                                                trials_csv_path, 
                                                mode='a', 
                                                header=not os.path.exists(trials_csv_path),
                                                index=False
                                            )
                                            
                                            # Update best results if better R2 found
                                            if r2 > best_r2:
                                                best_r2 = r2
                                                best_params = current_params
                                                best_model = trained_model
                                                best_metrics = {
                                                    'r2': r2,
                                                    'mse': mse,
                                                    'mae': mae
                                                }
                                                print("New best R2 score!")
                                                
                                                # Save the best model so far
                                                model_save_path = f"CNN_models/best_model_{dataset_name}.pth"
                                                os.makedirs('CNN_models', exist_ok=True)
                                                torch.save(best_model.state_dict(), model_save_path)
                                            
                                            # Check if target achieved
                                            if r2 >= target_r2:
                                                print(f"\nTarget R2 score of {target_r2} achieved!")
                                                break
                                            
            best_result = {
                'Dataset': data_file,
                'MSE': best_metrics['mse'],
                'MAE': best_metrics['mae'],
                'R2': best_metrics['r2'],
                'Units': str(best_params['num_units']),
                'Dropout': best_params['drop'],
                'Learning_Rate': best_params['lr'],
                'Weight_Decay': best_params['decay'],
                'Epochs': best_params['num_epochs'],
                'Activation': best_params['acts'],
                'Optimizer': best_params['opts'],
                'Patience': best_params['patience'],
                'Convolutions': best_params['blocks']
            }
            best_results_df = pd.concat([best_results_df, pd.DataFrame([best_result])], ignore_index=True)
            
  
            print("\nGrid search completed for this dataset!")
            print("\nBest parameters found:")
            for param, value in best_params.items():
                print(f"{param}: {value}")
            print(f"\nBest metrics:")
            print(f"R2: {best_metrics['r2']:.4f}")
            print(f"MSE: {best_metrics['mse']:.4f}") 
            print(f"MAE: {best_metrics['mae']:.4f}")
            
        except Exception as e:
            print(f"Error processing dataset {data_file}: {str(e)}")
            continue
    

    os.makedirs('CNN_results', exist_ok=True)
    best_results_df.to_csv("CNN_results/best_models_summary.csv", index=False)
    
    print("\nAll datasets processed. Best results saved to 'results/best_models_summary.csv'")
    

    best_overall_idx = best_results_df['R2'].idxmax()
    best_overall_dataset = best_results_df.loc[best_overall_idx, 'Dataset']
    print(f"\nBest overall model found for dataset: {best_overall_dataset}")
    print(f"R2 Score: {best_results_df.loc[best_overall_idx, 'R2']:.4f}")
    
    return best_params, best_model, best_metrics

In [14]:
best_params, best_model, best_metrics = grid_search_CNN(target_r2=0.9,data_folder="data")



Processing dataset: filtered_00001.csv
Total parameter combinations to try: 7776

Trial 1/7776
Current parameters: {'num_epochs': 250, 'num_units': [48, 24, 16, 8], 'drop': 0.1, 'lr': 0.001, 'decay': 0.0001, 'acts': 'relu', 'opts': 'adam', 'patience': 20, 'blocks': [32, 16, 16]}
R2 Score: 0.2847
MSE Score: 0.6145
MAE Score: 0.6669
New best R2 score!

Trial 2/7776
Current parameters: {'num_epochs': 250, 'num_units': [48, 24, 16, 8], 'drop': 0.1, 'lr': 0.001, 'decay': 1e-05, 'acts': 'relu', 'opts': 'adam', 'patience': 20, 'blocks': [32, 16, 16]}


C:\Users\LEGION\AppData\Local\Temp\ipykernel_2788\1428187504.py:148: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  all_trials_df = pd.concat([all_trials_df, pd.DataFrame([trial_results])], ignore_index=True)


R2 Score: 0.2936
MSE Score: 0.6069
MAE Score: 0.6538
New best R2 score!

Trial 3/7776
Current parameters: {'num_epochs': 250, 'num_units': [48, 24, 16, 8], 'drop': 0.1, 'lr': 0.0005, 'decay': 0.0001, 'acts': 'relu', 'opts': 'adam', 'patience': 20, 'blocks': [32, 16, 16]}
R2 Score: -2.6000
MSE Score: 3.0928
MAE Score: 1.5194

Trial 4/7776
Current parameters: {'num_epochs': 250, 'num_units': [48, 24, 16, 8], 'drop': 0.1, 'lr': 0.0005, 'decay': 1e-05, 'acts': 'relu', 'opts': 'adam', 'patience': 20, 'blocks': [32, 16, 16]}
R2 Score: -1.2334
MSE Score: 1.9187
MAE Score: 1.1443

Trial 5/7776
Current parameters: {'num_epochs': 250, 'num_units': [48, 24, 16, 8], 'drop': 0.1, 'lr': 0.0001, 'decay': 0.0001, 'acts': 'relu', 'opts': 'adam', 'patience': 20, 'blocks': [32, 16, 16]}
R2 Score: -6.7299
MSE Score: 6.6408
MAE Score: 2.4648

Trial 6/7776
Current parameters: {'num_epochs': 250, 'num_units': [48, 24, 16, 8], 'drop': 0.1, 'lr': 0.0001, 'decay': 1e-05, 'acts': 'relu', 'opts': 'adam', 'patienc

KeyboardInterrupt: 

In [15]:
PATH = "CNN_models/best_model_filtered_00001.pth"
model = createCNNmodel([32,16,16], [32,16,16,8], 0.3, 0.01, 1e-4, "relu", "adam")
model.load_state_dict(torch.load(PATH, weights_only=True))

all_preds = []
all_targets = []

test_loader = get_SNP("filtered_001.csv")
with torch.no_grad():
    for batch_X, batch_y in test_loader:
        batch_X = batch_X.to(device)
        outputs = model(batch_X)
        all_preds.append(outputs.cpu().numpy())
        all_targets.append(batch_y.numpy().reshape(-1, 1))  # Reshape to match predictions

y_pred = np.concatenate(all_preds).reshape(-1)
y_true = np.concatenate(all_targets).reshape(-1)

# Calculate metrics
mse = mean_squared_error(y_true, y_pred)
mae = mean_absolute_error(y_true, y_pred)
r2 = r2_score(y_true, y_pred)

print("\n=== Final Evaluation on Test Set ===")
print(f"MSE:  {mse:.4f}")
print(f"MAE:  {mae:.4f}")
print(f"R^2:  {r2:.4f}")

# Visualize results
plt.figure(figsize=(12, 5))

plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.title("Learning Curve (1D CNN)")
plt.xlabel("Epoch")
plt.ylabel("MSE Loss")
plt.legend()

plt.subplot(1, 2, 2)
plt.scatter(y_true, y_pred, alpha=0.5)
min_val = min(min(y_true), min(y_pred))
max_val = max(max(y_true), max(y_pred))
plt.plot([min_val, max_val], [min_val, max_val], 'r--')
plt.title("Predicted vs True Phenotype")
plt.xlabel("True Phenotype")
plt.ylabel("Predicted Phenotype")

plt.tight_layout()
plt.show()


TypeError: unsupported operand type(s) for //: 'str' and 'int'

In [ ]:
def importance_graph(model, snp_number=20):
    N = snp_number
    
    input_weights = model.state_dict()[list(model.state_dict().keys())[0]]
    
    snp_weights = input_weights.mean(dim=0).cpu().numpy()
    
    fig_height = max(5, N * 0.4)
    plt.figure(figsize=(12, fig_height))

    snp_names = np.array(X.columns)
    sorted_indices = np.argsort(snp_weights)

    plt.barh(snp_names[sorted_indices][-N:], snp_weights[sorted_indices][-N:], color="forestgreen", label="Positive Effect")
    plt.barh(snp_names[sorted_indices][:N], snp_weights[sorted_indices][:N], color="firebrick", label="Negative Effect")

    plt.xlabel("Feature Importance (Weight Value)")
    plt.ylabel("SNPs")
    plt.title("Top SNPs Affecting Phenotype")
    plt.gca().invert_yaxis()
    plt.legend()
    plt.savefig(f"important_{snp_number}_snps.png", dpi=300)

importance_graph(best_model, 20)